In [1]:
import tomica
print(tomica.__file__)

d:\tomica\tomica\__init__.py


In [2]:
import polars as pl
import random
import time
from datetime import datetime

## Usage

In [3]:
df = pl.read_csv('./data/csidata.csv')
df = df.with_columns(
    pl.col('datetime').str.strptime(pl.Datetime)
)
df

datetime,instrument,open,high,low,close,pre_close,change,pct_chg,amount,volume
datetime[μs],str,f64,f64,f64,f64,f64,f64,f64,f64,f64
2006-01-04 00:00:00,"""000001.SZ""",6.13,6.33,6.12,6.28,6.14,0.14,2.28,96989.0066,154450.68
2006-01-04 00:00:00,"""000002.SZ""",4.4,4.67,4.4,4.66,4.31,0.35,8.12,175187.0657,389310.43
2006-01-04 00:00:00,"""000012.SZ""",4.7,4.75,4.55,4.7,4.62,0.08,1.73,18863.8625,40399.61
2006-01-04 00:00:00,"""000016.SZ""",3.52,3.58,3.52,3.57,3.52,0.05,1.42,5131.8793,14426.65
2006-01-04 00:00:00,"""000021.SZ""",7.99,8.4,7.99,8.32,8.0,0.32,4.0,27657.1708,33526.6
…,…,…,…,…,…,…,…,…,…,…
2023-07-27 00:00:00,"""688363.SH""",94.34,96.2,94.0,95.9,94.7,1.2,1.2672,276386.76,28957.4
2023-07-27 00:00:00,"""688396.SH""",55.25,56.25,54.92,55.36,54.92,0.44,0.8012,218456.024,39268.57
2023-07-27 00:00:00,"""688561.SH""",48.96,49.38,48.8,48.83,49.11,-0.28,-0.5701,76213.86113,15532.4


## test speed

In [4]:
data = pl.select(
    pl.lit(1).sample(n=1e6, with_replacement=True)  # 先采样
    .map_elements(lambda _: random.uniform(-50, 50), return_dtype=pl.Float64)
    .alias("foo")
).to_series()

In [5]:
def test_speed(fn, window_size):
    st = time.time()
    try:
        fn(data, window_size)
        print(f'{fn.__name__:<20} {time.time()-st:>8.4f} s')
    except:
        fn(data, data, window_size)
        print(f'{fn.__name__:<20} {time.time()-st:>8.4f} s')

In [6]:
for func in tomica.operators.__all__:
    func = getattr(tomica.operators,func)
    test_speed(func,10)

ts_delay               0.6072 s
ts_delta               0.0157 s
ts_returns             0.0182 s
ts_sum                 0.0168 s
ts_product             0.0098 s
ts_max                 0.0262 s
ts_min                 0.0159 s
ts_mean                0.0237 s
ts_std                 0.0078 s
ts_rank                0.2236 s
ts_variance            0.0082 s
ts_quantile_up         0.0717 s
ts_quantile_down       0.0584 s
ts_zscore              0.0466 s
ts_robust_zscore       0.1415 s
ts_scale               0.0777 s
ts_sharpe              0.0502 s
ts_av_diff             0.0232 s
ts_min_diff            0.0369 s
ts_max_diff            0.0345 s
ts_cov                 0.0550 s
ts_beta                0.1265 s
ts_pos_count           0.0302 s
ts_neg_count           0.0380 s
decay_n                0.0195 s


In [21]:
st = time.time()
df['open'].rolling_rank(10)
print(f'{time.time()-st:.4f} s')

0.2344 s


In [20]:
st = time.time()
tomica.operators.ts_rank(df['open'],10)
print(f'{time.time()-st:.4f} s')

0.2430 s
